In [1]:
import json
import csv
import sklearn
import numpy as np
import random
from sklearn import preprocessing

Import environment data, and extract temprature and humidity

In [2]:
env_name = ['date','temperature','humidity']
env_index = [0,2,4]

env_data = {}

with open('KITH_2018-04-01_2018-12-31.csv') as env_data_file:
    data_reader2 = csv.reader(env_data_file, delimiter = ',')
    count = 0
    for row in data_reader2:
        if count == 0: #skip the header
            count += 1
            continue
        date = row[0]
        if date not in env_data and row[2]!='' and row[4]!='':
            env_data[date] = {}
            env_data[date]['temperature'] = []
            env_data[date]['humidity'] = []
        else:
            continue
            
        env_data[date]['temperature'].append(float(row[2]))
        env_data[date]['humidity'].append(float(row[4]))
            
#calculate the average for temperature and humidity during each day
env_data_avg = {}
for date in env_data:
    env_data_avg[date] = {}
    temp = np.array(env_data[date]['temperature']).mean()
    humid = np.array(env_data[date]['humidity']).mean()
    env_data_avg[date]['temperature'] = temp
    env_data_avg[date]['humidity'] =  humid
    #compute THI
    env_data_avg[date]['THI'] = temp * 0.8 + humid * (temp - 14.4)/100 + 46.4 
    
env_data_avg

{'4/1/18': {'temperature': 10.0, 'humidity': 52.0, 'THI': 52.111999999999995},
 '4/2/18': {'temperature': -1.7, 'humidity': 78.0, 'THI': 32.482},
 '4/3/18': {'temperature': -2.2, 'humidity': 88.0, 'THI': 30.031999999999996},
 '4/4/18': {'temperature': 5.0, 'humidity': 93.0, 'THI': 41.658},
 '4/5/18': {'temperature': -4.4, 'humidity': 88.0, 'THI': 26.336},
 '4/6/18': {'temperature': -7.8, 'humidity': 92.0, 'THI': 19.735999999999997},
 '4/7/18': {'temperature': -2.0, 'humidity': 100.0, 'THI': 28.4},
 '4/8/18': {'temperature': -5.6, 'humidity': 75.0, 'THI': 26.919999999999998},
 '4/9/18': {'temperature': -6.1, 'humidity': 74.0, 'THI': 26.349999999999998},
 '4/10/18': {'temperature': -1.1, 'humidity': 66.0, 'THI': 35.29},
 '4/11/18': {'temperature': -4.4, 'humidity': 96.0, 'THI': 24.831999999999997},
 '4/12/18': {'temperature': -2.8, 'humidity': 92.0, 'THI': 28.336000000000002},
 '4/13/18': {'temperature': 12.8, 'humidity': 69.0, 'THI': 55.536},
 '4/14/18': {'temperature': 2.8, 'humidity':

In [3]:
#read production data from production data csv file, store in both row and column format
#incorporate environment data for corresponding date

prod_name = ['animal_id','date','activity','prod_rate','yield','fat','protein','lactose']
prod_index = [0,2,14,17,7,9,10,11]
prod_row = [] #each entry in data_row is a dictionary/json object from a row in csv file
prod_column = {} #each key,val pair in data_column is the list of all data for a data field
for name in prod_name:
    prod_column[name] = []
prod_column['temperature'] = []
prod_column['humidity'] = []
prod_column['THI'] = []
    
with open('Production data.csv') as prod_data_file:
    data_reader = csv.reader(prod_data_file, delimiter = ',')
    for row in data_reader:
        #incorporate environment data; if no evironment data for the date, discard this row
        date = row[2]
        if date not in env_data_avg:
            continue
            
        new_row = {}
        temp = env_data_avg[date]['temperature']
        humid = env_data_avg[date]['humidity']
        thi = env_data_avg[date]['THI']
        prod_column['temperature'].append(temp)
        prod_column['humidity'].append(humid)
        prod_column['THI'].append(thi)
        new_row['temperature'] = temp
        new_row['humidity'] = humid
        new_row['THI'] = thi
        
        for i in range(len(prod_index)):
            idx = prod_index[i]
            name = prod_name[i]
            if i== 1:
                prod_column[name].append(str(row[idx]))
                new_row[name] = str(row[idx])
            else:
                prod_column[name].append(float(row[idx]))
                new_row[name] = float(row[idx])
                
        prod_row.append(new_row)

Since our goal is to predic milk yield and composition based on given cow_id and contemporary environment data,
we would want to train a model based on each cow. Therefore, re-process the data with key as cow_id

In [139]:
#count how many data does each cow has and return the unique animal_id
cow_idx, cow_count = np.unique(prod_column['animal_id'], return_counts = True)
cow_idx = [int(cow_idx[i].item()) for i in range(len(cow_idx))]
cow_count = [int(cow_count[i].item()) for i in range(len(cow_idx))]
cow_count_dic = {}
for i in range(len(cow_idx)):
    idx = cow_idx[i]
    cnt = cow_count[i]
    if cnt not in cow_count_dic:
        cow_count_dic[cnt] = []
    cow_count_dic[cnt].append(idx)

cow_prod_row = {}
for idx in cow_idx:
    cow_prod_row[idx] = []

for i in range(len(prod_row)):
    row_i = prod_row[i]
    idx = row_i['animal_id']
    cow_prod_row[idx].append(row_i) 

In [140]:
cow_dataNum = dict(zip(cow_idx, cow_count))

In [7]:
#convert to columns without preprocessing
cow_prod_col = {}
for idx in cow_idx:
    cols = RowToColumn(cow_prod_row[idx])
    cow_prod_col[idx] = cols

In [110]:
#data preprocessing: mean removal and variance scaling: data then has zero mean and unit variance
#we will do mean removal and variance scaling for each cow
#and before scaling, we need to record the mean and variance of data from each cow 
#so that later prediction results could be convert back to meaningful real data

cow_prod_stats = {}
cow_prod_scaled_col = {}
for idx in cow_idx:
    cow_prod_stats[idx] = {}
    cow_prod_scaled_col[idx] = {}

for idx in cow_idx:
    #convert the data for each cow to columns
    cols = RowToColumn(cow_prod_row[idx])
    #cow_prod_scaled_col[idx]['temperature'] = cols['temperature'] 
    #cow_prod_scaled_col[idx]['humidity'] = cols['humidity'] 
    #cow_prod_scaled_col[idx]['THI'] = cols['THI'] 
    for col in cols:
        if col == 'animal_id' or col == 'date':
        #if col == 'animal_id' or col == 'date' or col == 'temperature' or col=='humidity' or col=='THI':
            continue
        else:
            npcol = np.array(cols[col])
            #record stats for the current column
            cow_prod_stats[idx][col] = {}
            cow_prod_stats[idx][col]['mean'] = np.mean(npcol)
            cow_prod_stats[idx][col]['std'] = np.std(npcol)
            
            #do scaling 
            cow_prod_scaled_col[idx][col] = preprocessing.scale(npcol)


In [6]:
#utility functions

#rows: a list of json objects, each json object is a row of data
#columns: a json obejct, while each value is a list
#precondition: no missing data
def RowToColumn(rows):
    columns = {}
    for key in rows[0]:
        columns[key] = []
    for row in rows:
        for key in row:
            columns[key].append(row[key])
    return columns
    
def ColumnToRow(columns):
    rows = []
    n = 0
    for key in columns:
        n = len(columns[key])
        break
    for i in range(n):
        row_i = {}
        for key in columns:
            row_i[key] = columns[key][i]
        rows.append(row_i)
    return rows
        
#rows: a list of json/dictionary objects, each json object is a row of data
#columns: a json/dictionary obejct, while each value is a list

#return needed field values from each row 
def ExtractRowValues(rows,field):
    values = []
    for i in range(len(rows)):
        row_i = rows[i]
        value_i = []
        for key in field:
            value_i.append(row_i[key])
        values.append(value_i)
    return values

#return values from each column
def ExtractColumnValues(columns):
    values = []
    for key in columns:
        values.append(columns[key])
    return values
    

Start training data models

In [11]:
from sklearn import linear_model
from sklearn import model_selection
from sklearn import multioutput
from sklearn import svm
from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [9]:
#grab dataset from cow_col
#field: list of fields that are extracting from the dataset
def GetDataset(cow_id, field):
    data_col = cow_prod_col[cow_id]
    data_row = ColumnToRow(data_col)
    data_row = ExtractRowValues(data_row, field)
    return np.array(data_row)

In [15]:
#initialize regressors
def InitializeRegressors():
    reg1 = linear_model.LinearRegression(copy_X = True, fit_intercept = True) 
    reg2 = linear_model.ElasticNet(copy_X = True, fit_intercept = True)
    reg3 = linear_model.BayesianRidge(copy_X = True, fit_intercept = True)
    reg4 = linear_model.Ridge(copy_X = True, fit_intercept = True)
    
    return [reg1, reg2, reg3, reg4]

In [33]:
#linear regression
def Regression(dataset, i_start, i_end, o_start, o_end):
    X = dataset[:,i_start:i_end]
    Y = np.ravel(dataset[:,o_start:o_end])
    model_results = {}
    reg_names = ['least_square','elastic','bayesian','ridge']
    reg_scores_mean= []
    #reg_scores_std_min = 
    regs = InitializeRegressors()
    for i in range(len(regs)):
        reg = regs[i]
        reg_name = reg_names[i]
        num_k = int(len(X)/25)
        cv = KFold(num_k)
        #cross_validation
        scores = model_selection.cross_val_score(reg, X, Y, cv = cv, scoring = 'neg_mean_squared_error')
        reg_scores_mean.append(abs(np.mean(scores)))
        model_results[reg_name] = {'score':abs(np.mean(scores)), 'model':reg}
    #find and return the best model 
    min_scores_mean = np.argmin(reg_scores_mean)
    min_scores = model_results[reg_names[min_scores_mean]]['score']
    min_scores_reg = model_results[reg_names[min_scores_mean]]['model']
    #fit dataset to the selected model again
    min_scores_reg.fit(X,Y)
    return (min_scores, min_scores_reg)

In [103]:
#main function for training the relationship between environment and milk yield and components without activity
#return trained models stats
def TrainEnvYield():
    min_data = [5*(27-i) for i in range(18)] #different min data requirement, interval 5
    field = ['temperature','humidity','THI','yield','fat','protein','lactose']
    #store the trained models
    models = {}
    for minimum in min_data:
        cur_models = {}
        idxs = []
        for i in range(5): #all the cows that satisfy the minimum data number requirement
            if minimum+i in cow_count_dic:
                idxs.extend(cow_count_dic[minimum+i])
                
        for idx in idxs:
            cur_models[idx] = {}
            dataset = GetDataset(idx, field)
            #do linear regression for multiple regression models
            for i in range(7):
                if i<3: 
                    continue
                cur_models[idx][field[i]] = Regression(dataset,0,2,i,i+1)
            #print("trained_model results for cow_id:{} is {}".format(idx, models[idx]))
        models[minimum] = cur_models
    return models

In [104]:
trained_models = TrainEnvYield()

In [110]:
#output models to pickle files
#a new folder for each minimum data length
import pickle
import json
#stats = {}
for minimum in trained_models:
    min_model_name = "models/"+str(minimum)
    for idx in trained_models[minimum]:
        #stats[idx] = {}
        #stats[idx]['length'] = minimum
        filename = min_model_name+"/"+str(idx)+".pkl"
        with open(filename, 'wb') as model_file:
            for field in trained_models[minimum][idx]:
                pickle.dump(trained_models[minimum][idx][field][1], model_file)
                #stats[idx][field] = np.mean(cow_prod_col[idx][field])
#with open('model_stats.txt','wb') as stats_file:
    #json.dump(stats, stats_file)

In [141]:
#output stats to a json file 
import json
with open('model_stats.txt','w') as stats_file:
    json.dump(cow_dataNum, stats_file)

In [87]:
#returns None if no models exist for cow_id; otherwise return a tuple of length-4
#the tuple represents the predicted (yield, fat, protein, lactose) respectively
def GetModelAndPredict(cow_id, temperature, humidity):
    #see if there are models 
    with open('model_stats.txt','r') as stats_file:
        model_stats = json.load(stats_file)
    
    if str(cow_id) not in model_stats:
        return None
    
    #if there are models for cow_id, return predicted result
    fields = ['yield', 'fat','protein','lactose']
    results = []
    for field in fields:
        filename = "models/"+str(cow_id)+"-"+field+".pkl"
        with open(filename, 'rb') as model_file:
            loaded_model = pickle.load(model_file)
        results.append(loaded_model.predict([[temperature,humidity]])[0])
    return tuple(results)

In [20]:
dset = GetDataset(1, ['temperature','humidity','THI','yield','fat','protein','lactose'])
dset

array([[1.0600e+01, 8.6000e+01, 5.1612e+01, 3.5644e+04, 4.2200e+00,
        3.3200e+00, 4.4300e+00],
       [1.4400e+01, 7.8000e+01, 5.7920e+01, 3.6617e+04, 4.4600e+00,
        2.9000e+00, 4.7300e+00],
       [1.5000e+01, 6.7000e+01, 5.8802e+01, 3.2883e+04, 3.2300e+00,
        3.0200e+00, 4.3300e+00],
       [2.2200e+01, 7.8000e+01, 7.0244e+01, 4.3728e+04, 3.4000e+00,
        3.0700e+00, 4.7400e+00],
       [8.9000e+00, 9.3000e+01, 4.8405e+01, 4.8165e+04, 3.1800e+00,
        2.8900e+00, 4.6900e+00],
       [9.4000e+00, 8.6000e+01, 4.9620e+01, 4.7819e+04, 3.0700e+00,
        2.9600e+00, 4.6400e+00],
       [1.7800e+01, 9.6000e+01, 6.3904e+01, 4.5789e+04, 2.8500e+00,
        2.9800e+00, 4.5200e+00],
       [1.3300e+01, 9.0000e+01, 5.6050e+01, 4.8742e+04, 3.3600e+00,
        3.0700e+00, 4.4800e+00],
       [1.0600e+01, 9.6000e+01, 5.1232e+01, 4.9329e+04, 3.2600e+00,
        2.9200e+00, 4.6300e+00],
       [2.1100e+01, 9.0000e+01, 6.9310e+01, 5.1833e+04, 2.8800e+00,
        2.9400e+00, 4.5